In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

# Grupos

## Socios activos 2016

In [2]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [3]:
%%time

dtypes = {
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Edad': 'category',
}

perfiles_df = pd.read_csv(
    'valores_grupos_2016.csv', header=0, dtype=dtypes)
# perfiles_df = perfiles_df.fillna('')

CPU times: user 64.1 ms, sys: 14.1 ms, total: 78.2 ms
Wall time: 77.1 ms


In [4]:
perfiles_df.shape

(50272, 9)

In [5]:
perfiles_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
0,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,OTRO
1,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,51-64
2,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,OTRO
3,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,51-64
4,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,OTRO


# Agrupando valores frecuentes

### Grupos (I)

In [6]:
groups = [tuple(x) for i, x in perfiles_df.iterrows()]
counter = Counter(groups)

In [7]:
len(counter)

42621

In [8]:
counter.most_common(1)

[(('BUE',
   'sr.',
   'olimpico',
   'buenaventura',
   'CTA',
   'estudiante',
   'básica primaria',
   'OTRO',
   '15-'),
  75)]

##### => 50272 usuarios repartidos en 42621 grupos muy poco numerosos (sólo 75 usuarios el de mayor tamaño)

### Porcentajes sin valores

In [11]:
for c in sorted(dtypes):
    cns = perfiles_df[c].value_counts()
    total = cns.sum()
    otros = 0
    if 'OTRO' in cns:
        otros = cns.ix['OTRO']
    perc = '{:.2%}'.format(otros/total)
    print(c, perc)

Barrio 12.10%
CATEG 0.00%
Ciudad 5.60%
Edad 13.18%
Nivel Estudio1 12.68%
Ocupación1 3.64%
Sucursal 0.02%
Tratamiento 10.79%
Área de Estudio1 46.69%


##### => Eliminar columna Área de Estudio1 (46.69% sin valor)

### Sucursales y ciudades

In [118]:
sucursales_ciudades = {
    'ARM': 'armenia',
    'BAR': 'barranquilla',
    'BLAA': 'bogota',
    'BUE': 'buenaventura',
    'BUC': 'bucaramanga',
    'CAL': 'cali',
    'CAR': 'cartagena',
    'CASAGO': 'bogota',
    'CATMAN': 'bogota',
    'CUC': 'cucuta',
    'FLO': 'florencia',
    'GIR': 'girardot',
    'HON': 'honda',
    'IBA': 'ibague',
    'IEC': 'bogota',
    'IPI': 'ipiales',
    'LET': 'leticia',
    'MAN': 'manizales',
    'MED': 'medellin',
    'MON': 'monteria',
    'NEI': 'neiva',
    'PAS': 'pasto',
    'PER': 'pereira',
    'POP': 'popayan',
    'QUI': 'quibdo',
    'RIO': 'riohacha',
    'SAN': 'san andres',
    'SIN': 'sincelejo',
    'STM': 'santa marta',
    'TUN': 'tunja',
    'VAL': 'valledupar',
    'VIL': 'villavicencio',
}


In [133]:
p = q = r = 0

for i, x in perfiles_df.iterrows():
    s = x['Sucursal']
    cs = sucursales_ciudades.get(s, 'OTRO')
    c = x['Ciudad']
    if cs == c =='OTRO':
        p += 1
    elif cs == c:
        q += 1
    else:
        r += 1

print(p, q, r)
print('{:.2%}'.format((p + q) / (p + q + r)))

8 44290 5974
88.12%


##### => Eliminar columna Ciudad (88.12% duplicados)

### Grupos (II)

In [14]:
final_columns = ['Sucursal', 'Tratamiento', 'Barrio', 'CATEG', 'Ocupación1', 'Nivel Estudio1', 'Edad',]

In [16]:
perfiles_df = perfiles_df[final_columns].copy()

In [17]:
groups = [tuple(x) for i, x in perfiles_df.iterrows()]
counter = Counter(groups)

In [18]:
len(counter)

36382

In [19]:
counter.most_common(1)

[(('BLAA',
   'sr.',
   'OTRO',
   'CIS',
   'estudiante',
   'universitario 1 nivel',
   '16-28'),
  117)]

##### => 50272 usuarios repartidos en 36382 grupos muy poco numerosos (sólo 117 usuarios el de mayor tamaño)

### Combinaciones

In [70]:
def combinations(ls, ini=[], together=[]):
    def basic_combinations(ls):
        combs = []
        for i in range(1, len(ls) + 1):
            combs.append(itertools.combinations(ls, i))
        combs = [c for comb in combs for c in comb]
        return combs
    
    combs = basic_combinations(ls)
    
    aux = list(combs)
    for e in ini:
        for c in combs:
            if e not in c:
                if c in aux:
                    aux.remove(c)
    
    result = list(aux)
    for x, y in together:
        for c in aux:
            if x in c and y not in c or x not in c and y in c:
                if c in aux:
                    result.remove(c)
    
    return result

In [71]:
final_columns

['Sucursal',
 'Tratamiento',
 'Barrio',
 'CATEG',
 'Ocupación1',
 'Nivel Estudio1',
 'Edad']

In [76]:
combs = combinations(final_columns, ini=['Tratamiento', 'Edad'], together=[('Sucursal', 'Barrio')])

In [77]:
combs

[('Tratamiento', 'Edad'),
 ('Tratamiento', 'CATEG', 'Edad'),
 ('Tratamiento', 'Ocupación1', 'Edad'),
 ('Tratamiento', 'Nivel Estudio1', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'Edad'),
 ('Tratamiento', 'CATEG', 'Ocupación1', 'Edad'),
 ('Tratamiento', 'CATEG', 'Nivel Estudio1', 'Edad'),
 ('Tratamiento', 'Ocupación1', 'Nivel Estudio1', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'CATEG', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'Ocupación1', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'Nivel Estudio1', 'Edad'),
 ('Tratamiento', 'CATEG', 'Ocupación1', 'Nivel Estudio1', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'CATEG', 'Ocupación1', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'CATEG', 'Nivel Estudio1', 'Edad'),
 ('Sucursal', 'Tratamiento', 'Barrio', 'Ocupación1', 'Nivel Estudio1', 'Edad'),
 ('Sucursal',
  'Tratamiento',
  'Barrio',
  'CATEG',
  'Ocupación1',
  'Nivel Estudio1',
  'Edad')]